In [32]:
import wandb
from src.models.TAE import TAE
from src.datasets.polynomial import PolynomialDataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.cluster import AgglomerativeClustering

In [2]:
run = wandb.init(project='DTC', name='test')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tristanbester1. Use `wandb login --relogin` to force relogin


In [3]:
model_artifact = run.use_artifact('tristanbester1/DTC/TAE-3qwd8j4j:v999', type='model')
model_artifact_dir = model_artifact.download()

In [4]:
model = TAE(
        input_dim=1,
        seq_len=100,
        cnn_kernel=10,
        cnn_stride=3,
        mp_kernel=10,
        mp_stride=3,
        lstm_hidden_dim=8,
        upsample_scale=2,
        deconv_kernel=10,
        deconv_stride=6,
    )

In [5]:
model.load_state_dict(torch.load('/Users/tristan/Documents/CS/Research/DTC/artifacts/TAE-3qwd8j4j:v999/model.pt', map_location=torch.device('cpu')))    

<All keys matched successfully>

In [6]:
dataset = PolynomialDataset(
    run,
    "tristanbester1/DTC/polynomial_dataset_X:v0",
    "tristanbester1/DTC/polynomial_dataset_Y:v0",
)

In [7]:
data_loader = DataLoader(dataset, 1)

In [131]:
preds = []

for x, _ in data_loader:
    pred = model(x)
    preds.append(pred.detach())

In [132]:
x = torch.cat(preds)

In [133]:
x.shape

torch.Size([300, 100, 1])

In [140]:
x = torch.tensor([
    [[0],[0],[0]],
    [[1],[1],[1]],
    [[2],[2],[2]],
    [[3],[3],[3]],
])
batch_size = 4
n_time_steps = 3
n_dims = 1

In [141]:
x.shape

torch.Size([4, 3, 1])

In [142]:
x.squeeze(-1)

tensor([[0, 0, 0],
        [1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])

In [145]:
a = x.squeeze(-1).repeat(batch_size,1,1)

In [146]:
a

tensor([[[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3]],

        [[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3]],

        [[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3]],

        [[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3]]])

In [156]:
a.shape

torch.Size([4, 4, 3])

In [152]:
x.squeeze(-1).repeat(1, batch_size, 1)

tensor([[[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3],
         [0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3],
         [0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3],
         [0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3]]])

In [162]:
x.squeeze(-1).repeat(1, batch_size, 1).reshape(batch_size, batch_size, n_time_steps)

tensor([[[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3]],

        [[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3]],

        [[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3]],

        [[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3]]])

In [179]:
b = x.repeat(1,1,batch_size).squeeze(-1).reshape(batch_size,batch_size,n_time_steps)

In [180]:
a

tensor([[[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3]],

        [[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3]],

        [[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3]],

        [[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2],
         [3, 3, 3]]])

In [181]:
b

tensor([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],

        [[2, 2, 2],
         [2, 2, 2],
         [2, 2, 2],
         [2, 2, 2]],

        [[3, 3, 3],
         [3, 3, 3],
         [3, 3, 3],
         [3, 3, 3]]])

In [185]:
((a - b) ** 2).sum(dim=2)

tensor([[ 0,  3, 12, 27],
        [ 3,  0,  3, 12],
        [12,  3,  0,  3],
        [27, 12,  3,  0]])

In [ ]:
x

In [29]:
X = preds

In [45]:
def euclidean(x, y):
    return torch.sum((x-y) ** 2)

In [81]:
x = torch.tensor([
    [[0,0,0]],
    [[1,1,1]],
    [[2,2,2]],
])

In [98]:
x.shape

torch.Size([3, 1, 3])

In [108]:
x_1 = x.repeat(1, 3, 1).reshape(9, 1, 3)

In [109]:
x_2 = x.repeat(3, 1, 1)

In [113]:
torch.sum((x_1 - x_2) ** 2, dim=2).reshape(3,3)

tensor([[ 0,  3, 12],
        [ 3,  0,  3],
        [12,  3,  0]])

In [186]:
print(torch.sum((x[0] - x[0]) ** 2))
print(torch.sum((x[0] - x[1]) ** 2))
print(torch.sum((x[0] - x[2]) ** 2))
print(torch.sum((x[0] - x[3]) ** 2))

tensor(0)
tensor(3)
tensor(12)
tensor(27)


In [83]:
t_1 = x[0]

In [90]:
torch.sum((t_1.expand(3, 1, 3) - x) ** 2, dim=2)

tensor([[ 0],
        [ 3],
        [12]])

In [82]:
x - x

tensor([[[0, 0, 0]],

        [[0, 0, 0]],

        [[0, 0, 0]]])

In [74]:
z = x.expand((x.shape[0], x.shape[0], 3))

In [75]:
mse = torch.sqrt(torch.sum((z - x.unsqueeze(1)) ** 2, dim=2))

In [76]:
mse

tensor([[[0.0000, 0.0000, 0.0000],
         [1.4142, 1.4142, 1.4142]],

        [[1.4142, 1.4142, 1.4142],
         [0.0000, 0.0000, 0.0000]]])

In [77]:
torch.transpose(mse, 0, 1)

tensor([[[0.0000, 0.0000, 0.0000],
         [1.4142, 1.4142, 1.4142]],

        [[1.4142, 1.4142, 1.4142],
         [0.0000, 0.0000, 0.0000]]])

In [33]:
cluster_algo = AgglomerativeClustering(
    n_clusters=3,
    affinity='euclidean',
    linkage='complete',
)

In [34]:
cluster_algo.fit_predict(X)

ValueError: Found array with dim 3. AgglomerativeClustering expected <= 2.